In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import praw
import time
import datetime
from datetime import datetime, timedelta

In [2]:
client = MongoClient('localhost', 27017)
db = client.db
headlines = db.headlines
threads = db.threads
analysis = db.analysis

In [3]:
df = pd.read_csv('companylist.csv')

In [4]:
secretfile = open('secret.txt')
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secretfile.read(), user_agent='sentiment')

In [5]:
wantedTags = ['Daily Discussion', 'DD', 'Technicals', 'Fundamentals', 'Discussion', 'YOLO', 'Stocks']
subreddit = reddit.subreddit('wallstreetbets')
postlist = []

for post in subreddit.hot(limit = 1000):
    if datetime.today().weekday == 0:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)
            
            currentTime = datetime.utcnow()
            
            postDelta = currentTime - postDate

            if postDelta.days < 3 and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break
    
    else:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)

            currentTime = datetime.utcnow()

            postDelta = currentTime - postDate
            postDelta = str(postDelta)

            if 'day' not in postDelta and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break

In [6]:
postDf = pd.DataFrame(columns = ['Date','Title','Body'])

In [7]:
dates = '{}/{}/{}'.format(currentTime.month,currentTime.day,currentTime.year)
titles = [post.title for post in postlist]
bodies = [post.selftext for post in postlist]
tags = [post.link_flair_text for post in postlist]

In [8]:
postDf['Title'] = titles
postDf['Body'] = bodies
postDf['Date'] = dates
postDf['Tag'] = tags

In [9]:
stocklist = df['Symbol']

In [10]:
stocklist = [s for s in stocklist if s.isalpha()]
stocklistcopy = ['$'+ s for s in stocklist]
stocklist = [s for s in stocklist if len(s) >= 3]
stocklist = stocklist + stocklistcopy

In [11]:
def findStocks(string):
    stockset = set(string.split()).intersection(stocklist)
    return stockset

In [12]:
postDf['set1'] = postDf['Body'].apply(findStocks)
postDf['set2'] = postDf['Title'].apply(findStocks)
postDf['Stocks'] = postDf.apply(lambda x: x['set1'].union(x['set2']), axis = 1)
postDf = postDf.drop(columns = ['set1','set2'])

In [13]:
postDf

,Date,Title,Body,Tag,Stocks
0,9/21/2020,Great write-up debunking NKLA (not mine),Well some fancy smarty pants over at r/investi...,DD,{NKLA}
1,9/21/2020,Costco earnings Thursday - don’t miss out!,The expectation is Costco will smash earnings ...,DD,{}
2,9/21/2020,Play money to house down-payment money,"Greetings Fellow Autists,\n\nLet's say, hypoth...",Discussion,"{TSLA, TTD, AAPL}"
3,9/21/2020,Taco bell wine -> CLX,"Imagine pounding down 6 soft tacos, 2 choco ta...",DD,{CLX}
4,9/21/2020,Why Dropbox ($DBX) might be a bargain right now.,"As you might know, Dropbox is one of the bigge...",DD,{}
5,9/21/2020,$TSLA to the moon,I've read a lot of attempts to decrypt Elons t...,DD,{$TSLA}
6,9/21/2020,Anybody else holding JWN Nordstrom?,Whenever I visit a Nordstrom these days it’s p...,Stocks,{JWN}
7,9/21/2020,Post-Covid play (Zimmer Biomet),I was thinking about backlogs that Covid-19 m...,Stocks,{ZBH}
8,9/21/2020,The best DD you will get around here,"Battery day is upon us my children, as I sit h...",DD,{}
9,9/21/2020,1 Month Market Forecast Using Palantiri Orb,"What’s up retards, I got early access to Palan...",Discussion,"{QQQ, $NVTA}"


In [14]:
df['Name'].unique().size

4790

In [15]:
unwantedEndings = ['INC.', 'CORP.', 'CORPORATION', 'COMPANY', 'INC', 'L.P.', 'LLC']

In [16]:
df

,Symbol,Name,LastSale,MarketCap,Country,IPOyear,Sector,Industry
0,TXG,"10x Genomics, Inc.",121.8900,1.272031e+10,United States,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments
1,PIH,"1347 Property Insurance Holdings, Inc.",3.9700,2.409038e+07,United States,2014.0,Finance,Property-Casualty Insurers
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.9999,0.000000e+00,United States,NaN,Finance,Property-Casualty Insurers
3,TURN,180 Degree Capital Corp.,1.9702,6.131570e+07,United States,NaN,Finance,Finance/Investors Services
4,FLWS,"1-800-FLOWERS.COM, Inc.",23.0400,1.484037e+09,United States,1999.0,Consumer Services,Other Specialty Stores
...,...,...,...,...,...,...,...,...
5716,ZUMZ,Zumiez Inc.,30.0100,7.636060e+08,United States,2005.0,Consumer Services,Clothing/Shoe/Accessory Stores
5717,ZYME,Zymeworks Inc.,45.3400,2.068740e+09,Canada,2017.0,Health Care,Major Pharmaceuticals
5718,ZYNE,"Zynerba Pharmaceuticals, Inc.",3.7000,1.082441e+08,United States,2015.0,Health Care,Major Pharmaceuticals
5719,ZYXI,"Zynex, Inc.",17.2000,5.969356e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...


In [24]:
test = df.sort_values(by = ['MarketCap'], ascending = False).reset_index()

In [26]:
test.loc[test['Symbol'] == 'COST']

,index,Symbol,Name,LastSale,MarketCap,Country,IPOyear,Sector,Industry
39,1322,COST,Costco Wholesale Corporation,335.96,1.483340e+11,United States,NaN,Consumer Services,Department/Specialty Retail Stores
